<a href="https://colab.research.google.com/github/juno-yuu/COVID-Vaccine-and-Biotech-industry/blob/Erin/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

In [6]:
!git clone https://github.com/juno-yuu/COVID-Vaccine-and-Biotech-industry.git

Cloning into 'COVID-Vaccine-and-Biotech-industry'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [9]:
data = pd.read_csv("/content/COVID-Vaccine-and-Biotech-industry/us_state_vaccinations.csv")

In [11]:
data.describe()

,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
count,7.878000e+03,7.747000e+03,7.735000e+03,7260.000000,7362.000000,7.653000e+03,7337.00000,7349.000000,8.083000e+03,8.083000e+03,7552.000000,7747.000000
mean,4.065853e+06,5.265809e+06,2.557398e+06,15.151172,38.762373,1.551882e+06,24.31409,52.035727,6.839717e+04,6.720646e+04,6202.969809,0.715817
std,1.873494e+07,2.423850e+07,1.168513e+07,11.781927,25.586672,7.722812e+06,14.69138,31.480798,2.869498e+05,2.733897e+05,2948.195674,0.149996
min,4.160000e+02,6.000000e+03,4.010000e+02,0.000000,0.170000,1.000000e+00,0.24000,6.140000,-4.453500e+04,-6.780000e+02,-499.000000,0.043000
25%,2.794162e+05,4.221500e+05,1.952275e+05,4.577500,15.975000,6.592800e+04,11.41000,21.880000,4.018500e+03,6.479500e+03,4040.500000,0.649000
50%,9.673035e+05,1.271990e+06,6.267010e+05,12.930000,35.445000,3.122400e+05,22.61000,48.110000,1.670400e+04,1.882600e+04,5974.000000,0.755000
75%,2.714834e+06,3.450202e+06,1.636262e+06,24.370000,59.260000,9.937900e+05,35.55000,78.830000,4.569900e+04,4.528650e+04,7949.000000,0.812000
max,2.735452e+08,3.445035e+08,1.574856e+08,55.660000,134.690000,1.232827e+08,84.87000,137.380000,4.629928e+06,3.384387e+06,71744.000000,1.201000
